# Modules

In [142]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import random as rnd
from tqdm import trange

# Functions

In [182]:
def Ebert_Reviews(num_pages=1):
    main_url = 'https://www.rogerebert.com'
    contr = '/contributors/roger-ebert/reviews/page/'
    movie_list = list()
    
    def Scrape_Review(href):
        rev_page = requests.get(main_url + href).text
        soup = BeautifulSoup(rev_page, 'html.parser')
        review = soup.find('div', {'itemprop': 'reviewBody'}).find_all('p')
        text = ' '.join(par.text for par in review if par.text != 'Advertisement').encode('utf-8').strip()
        # a few movies do not have rating
        try:
            rating = float(soup.find('meta', {'itemprop': 'ratingValue'}).get('content'))
        except:
            rating = np.nan
        return rating, text

    for i in trange(0,num_pages):
        contr_page = requests.get(main_url + contr + str(i+1)).text
        soup = BeautifulSoup(contr_page, 'html.parser')
        movies = soup('figure', {'class', 'movie review'})
        
        for movie in movies:
            time.sleep(rnd.random()) # be gentle and wait...
            title = movie.find_all('a')[1].text.encode('utf-8').strip()
            try: 
                year = int(movie.find('span', {'class':'release-year'}).text[1:-1])
            except:
                year = np.nan
                
            href = movie.a.get('href')
            rating, text = Scrape_Review(href)
                
            movie_list.append([title, year, href, rating, text])
        
    df = pd.DataFrame(movie_list, columns = ['title', 'year', 'url', 'rating', 'review'])
    return df 

In [183]:
dataframe = Ebert_Reviews(319)
#dataframe.to_csv('Ebert.csv', index = False)

100%|██████████| 1/1 [00:16<00:00, 16.60s/it]


In [185]:
dataframe.head()

,title,year,url,rating,review
0,The Spectacular Now,2013,/reviews/the-spectacular-now-2013,4.0,[Editor's note: Roger Ebert filed this review ...
1,Computer Chess,2013,/reviews/computer-chess-2013,2.0,[Editor's note: Roger Ebert filed this review ...
2,At Any Price,2012,/reviews/at-any-price-2012,4.0,"Ramin Bahrani, the best new American director ..."
3,Blancanieves,2012,/reviews/blancanieves-2012,4.0,Note: The following was reworked from a blog p...
4,Deceptive Practice: The Mysteries and Mentors ...,2013,/reviews/deceptive-practice-the-mysteries-and-...,3.0,This is another of Roger Ebert's final reviews...
